In [1]:
import psycopg2
import random
import csv
from faker import Faker

In [2]:
fake = Faker()

In [3]:
def get_connection():
    """Establish and return a database connection."""
    return psycopg2.connect(
        host="localhost",
        database="5310 project",
        user="postgres",
        password="123"
    )

In [4]:
def generate_product_warehouse(n=1000):
    """Generate and insert data into the product_warehouse table and save it to a CSV file."""
    try:
        conn = get_connection()
        cursor = conn.cursor()
        
        # Fetch product_ids and warehouse_ids from the respective tables
        cursor.execute("SELECT product_id FROM products")
        products = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT warehouse_id FROM warehouses")
        warehouses = [row[0] for row in cursor.fetchall()]

        if not products or not warehouses:
            print("Error: No products or warehouses found in the database.")
            cursor.close()
            conn.close()
            return

        # Ensure that each product is assigned to exactly one warehouse
        product_warehouse_data = []
        used_products = set()
        used_warehouses = set()
        
        while len(product_warehouse_data) < n and len(used_products) < len(products):
            product_id = random.choice(products)
            if product_id in used_products:
                continue  # Skip if product already assigned to a warehouse
            
            warehouse_id = random.choice(warehouses)
            
            # Ensure warehouse is not empty (to avoid the scenario where the warehouse might end up with no products)
            if not used_warehouses or len(used_warehouses) == len(warehouses):
                warehouse_id = random.choice(warehouses)
            
            
            # Append data to the list
            product_warehouse_data.append((product_id, warehouse_id))
            used_products.add(product_id)
            used_warehouses.add(warehouse_id)
            
            # Insert data into the database
            cursor.execute(
                "INSERT INTO product_warehouse (product_id, warehouse_id) VALUES (%s, %s)",
                (product_id, warehouse_id)
            )

        conn.commit()
        cursor.close()
        conn.close()

        # Write the generated product_warehouse data to a CSV file
        with open('C:/Users/Yihua/Desktop/5310 project/product_warehouse.csv', 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['product_id', 'warehouse_id'])
            for data in product_warehouse_data:
                csv_writer.writerow(data)

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if conn:
            cursor.close()
            conn.close()

if __name__ == "__main__":
    generate_product_warehouse()